In [0]:
-- Create schema inside your catalog
CREATE SCHEMA IF NOT EXISTS sales.dbo;

-- Set current catalog and schema context
USE CATALOG sales;
USE SCHEMA dbo;

-- Create products table
CREATE OR REPLACE TABLE sales.dbo.products (
  product_id INT,
  category STRING,
  product_name STRING
) USING DELTA;

-- Create sales_data table
CREATE OR REPLACE TABLE sales.dbo.sales_data (
  sale_id INT,
  product_id INT,
  sale_date DATE,
  quantity INT,
  unit_price DOUBLE
) USING DELTA;

-- Create or replace view with logic
CREATE OR REPLACE VIEW sales.dbo.sales_summary_view AS
SELECT
  DATE_TRUNC('month', s.sale_date) AS sale_month,
  p.category,
  SUM(s.quantity) AS total_quantity_sold,
  SUM(s.quantity * s.unit_price) AS total_revenue
FROM sales_data s
JOIN products p ON s.product_id = p.product_id
WHERE s.quantity > 0 AND s.unit_price > 0
AND s.sale_date >= DATEADD(day, -30, current_date())
GROUP BY DATE_TRUNC('month', s.sale_date), p.category;

-- Create target table if not exists
CREATE TABLE IF NOT EXISTS sales.dbo.monthly_sales_summary (
  sale_month DATE,
  category STRING,
  total_quantity_sold BIGINT,
  total_revenue DOUBLE
) USING DELTA;

-- Incremental merge (upsert)
MERGE INTO sales.dbo.monthly_sales_summary AS target
USING sales.dbo.sales_summary_view AS source
ON target.sale_month = source.sale_month AND target.category = source.category
WHEN MATCHED THEN UPDATE SET
  target.total_quantity_sold = source.total_quantity_sold,
  target.total_revenue = source.total_revenue
WHEN NOT MATCHED THEN INSERT (
  sale_month, category, total_quantity_sold, total_revenue
) VALUES (
  source.sale_month, source.category, source.total_quantity_sold, source.total_revenue
);


